In [1]:
!pip install fasttext
!pip install fastparquet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 68.8/68.8 kB 684.0 kB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Using cached pybind11-2.12.0-py3-none-any.whl (234 kB)
  Created wheel for fasttext: filename=fasttext-0.9.2-cp310-cp310-linux_x86_64.whl size=4227140 sha256=0796c15c998cf1a918e111fe685eaed0b890081b83e6284544d247df9f3eef67
  Stored in directory: /root/.cache/pip/wheels/a5/13/75/f811c84a8ab36eedbaef977a6a58a98990e8e0f1967f98f394
Successfully built fasttext
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 8.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 38.7 MB/s eta 0:00:00


In [2]:
from transformers import AutoTokenizer, AutoModel, AutoModelForSeq2SeqLM
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset,DataLoader, random_split

In [4]:
test_data = pd.read_parquet('test-00000-of-00001-47685aa42db61e77.parquet', engine='fastparquet')
train_data = pd.read_parquet('train-00000-of-00001-210cfe9263b99806.parquet', engine='fastparquet')
valid_data = pd.read_parquet('valid-00000-of-00001-cc552de6d1a6fa4b.parquet', engine='fastparquet')
test_data

,id,query,answer,choices,gold
0,MedNLI0,\nTASK: Please classify the relationship betwe...,entailment,"[entailment, contradiction, neutral]",0
1,MedNLI1,\nTASK: Please classify the relationship betwe...,contradiction,"[entailment, contradiction, neutral]",1
2,MedNLI2,\nTASK: Please classify the relationship betwe...,neutral,"[entailment, contradiction, neutral]",2
3,MedNLI3,\nTASK: Please classify the relationship betwe...,entailment,"[entailment, contradiction, neutral]",0
4,MedNLI4,\nTASK: Please classify the relationship betwe...,contradiction,"[entailment, contradiction, neutral]",1
...,...,...,...,...,...
1417,MedNLI1417,\nTASK: Please classify the relationship betwe...,contradiction,"[entailment, contradiction, neutral]",1
1418,MedNLI1418,\nTASK: Please classify the relationship betwe...,neutral,"[entailment, contradiction, neutral]",2
1419,MedNLI1419,\nTASK: Please classify the relationship betwe...,entailment,"[entailment, contradiction, neutral]",0
1420,MedNLI1420,\nTASK: Please classify the relationship betwe...,contradiction,"[entailment, contradiction, neutral]",1


In [5]:
def find_pre_and_hyp(query):
    start_pre = query.find("[PRE]") + len("[PRE]")
    end_pre = query.find("[HYP]")
    start_hyp = query.find("[HYP]") + len("[HYP]")
    end_hyp = query.find("OUTPUT:")
    premise = query[start_pre:end_pre].strip()
    hypothesis = query[start_hyp:end_hyp].strip()

    return premise,hypothesis

In [6]:
class CustomDataset(Dataset):
    def __init__(self, x_list, y_list):
        self.samples = []
        for x,y in zip(x_list,y_list):
            x_tensor = torch.tensor(x,dtype = torch.float32)
            y_tensor = torch.tensor(y,dtype = torch.float32)
            self.samples.append((x_tensor,y_tensor))

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx):
        return self.samples[idx]

In [7]:
tokenizer = AutoTokenizer.from_pretrained("gsarti/biobert-nli")
sen2vec_model = AutoModel.from_pretrained("gsarti/biobert-nli")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/136 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/454 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/433M [00:00<?, ?B/s]

In [8]:
x_list = []
y_list = []
for query,answer in zip(train_data['query'],train_data['answer']):
    if answer == 'entailment':
       y = [1,0,0]
    elif answer == 'neutral':
       y = [0,1,0]
    elif answer == 'contradiction':
       y = [0,0,1]
    else:
        print('should not get here')

    premise,hypothesis = find_pre_and_hyp(query)
    s1 = tokenizer(premise,return_tensors="pt")
    s2 = tokenizer(hypothesis,return_tensors="pt")
    vec_1 = sen2vec_model(**s1)
    vec_2 = sen2vec_model(**s2)
    vecc_1 = vec_1[1][0].detach().numpy()
    vecc_2 = vec_2[1][0].detach().numpy()
    x = np.concatenate((vecc_1, vecc_2))

    x_list.append(x)
    y_list.append(y)

In [9]:
train_dataset = CustomDataset(x_list,y_list)

In [10]:
class NN(nn.Module):
    def __init__(self):
        super(NN, self).__init__()
        self.fc1 = nn.Linear(1536, 200)
        self.bn1 = nn.BatchNorm1d(200)
        self.fc2 = nn.Linear(200, 100)
        self.bn2 = nn.BatchNorm1d(100)
        self.fc3 = nn.Linear(100, 50)
        self.bn3 = nn.BatchNorm1d(50)
        self.fc4 = nn.Linear(50, 3)

    def forward(self, x):
        x = torch.tanh(self.fc1(x))
        x = self.bn1(x)
        x = torch.tanh(self.fc2(x))
        x = self.bn2(x)
        x = torch.tanh(self.fc3(x))
        x = self.bn3(x)
        x = nn.functional.softmax(self.fc4(x))
        return x

In [11]:
#transform = transforms.Compose([
    #transforms.ToTensor()
#])

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=1024, shuffle=True)
# test_loader = torch.utils.data.DataLoader(test_data, batch_size=64, shuffle=True)

In [12]:
model = NN()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [13]:
model.train()
num_epochs = 60
for epoch in range(num_epochs):
    for sente, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(sente)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

    print(loss.item())

print('Training complete.')

<ipython-input-10-9fbe914b7e43>:19: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  x = nn.functional.softmax(self.fc4(x))


0.9125746488571167
0.8784293532371521
0.8796853423118591
0.8614419102668762
0.8423248529434204
0.8233136534690857
0.8333194255828857
0.810191810131073
0.790290892124176
0.7910229563713074
0.7925787568092346
0.7800929546356201
0.7722280621528625
0.7779907584190369
0.771453857421875
0.7398016452789307
0.7595598101615906
0.7423122525215149
0.750789999961853
0.718791663646698
0.7156513929367065
0.7105984091758728
0.718399167060852
0.7004064917564392
0.7085881233215332
0.7030244469642639
0.6880437731742859
0.6906741261482239
0.6907971501350403
0.6883136630058289
0.7052344083786011
0.6629883050918579
0.6835325956344604
0.6769704222679138
0.6810709238052368
0.6625747680664062
0.6750601530075073
0.6609047055244446
0.6563316583633423
0.6666638255119324
0.657619833946228
0.6574493050575256
0.6590582132339478
0.6578025221824646
0.6429007649421692
0.6519090533256531
0.6425556540489197
0.6467135548591614
0.6609349250793457
0.6567245721817017
0.6598114371299744
0.647828996181488
0.6476343870162964
0

In [14]:
count = 0
r = 0
model.eval()
for query,answer in zip(test_data['query'],test_data['answer']):
    count = count + 1
    premise,hypothesis = find_pre_and_hyp(query)

    s1 = tokenizer(premise,return_tensors="pt")
    s2 = tokenizer(hypothesis,return_tensors="pt")
    vec_1 = sen2vec_model(**s1)
    vec_2 = sen2vec_model(**s2)
    vecc_1 = vec_1[1][0].detach().numpy()
    vecc_2 = vec_2[1][0].detach().numpy()
    x = np.concatenate((vecc_1, vecc_2))
    x_tensor = torch.tensor(x,dtype = torch.float32).unsqueeze(1).expand(-1,2)
    x_tensor = torch.transpose(x_tensor,0,1)


    a = np.argmax(model(x_tensor).detach().numpy()[0])
    if a == 0:
       an = 'entailment'
    elif a == 1:
       an = 'neutral'
    elif a == 2:
       an = 'contradiction'
    else:
       print('should not get  here')

    if an == answer:
       r = r + 1

print(r/count)

<ipython-input-10-9fbe914b7e43>:19: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  x = nn.functional.softmax(self.fc4(x))


0.6518987341772152


In [ ]:
torch.save(model,'68.84accuracy.pth')